In [ ]:
from improc.utils.experiments import loader, dataset, tags
from improc.processes import brightfield

import tifffile
import pathlib
import os

%run utils/experiment_chooser.ipynb
experiment_chooser.choose()

In [ ]:
experiment = experiment_chooser.fetch()
experiment_base = experiment_chooser.fetch_base()

OUTPUT_PATH = experiment_base / "processed_imgs" / "brightfield_preprocessed"
os.makedirs(OUTPUT_PATH, exist_ok=True)

processed = experiment.datasets[1]
stitched = list(dataset.select_by_tags(processed.images, tags.Mosaic))
stitched_not_stacked = list(dataset.filter_tags(stitched, tags.Stack))
for idx, orig in enumerate(stitched_not_stacked):
    preprocessed = brightfield.ineuron_preprocess(orig.load())
    vertex: tags.Vertex = orig.get_tag(tags.Vertex)
    channel: tags.Channel = orig.get_tag(tags.Channel)
    time: tags.Timepoint = orig.get_tag(tags.Timepoint)
    tifffile.imwrite(OUTPUT_PATH / f"{vertex.label}-{channel.value}-{time.index}.tif", preprocessed)
    print("\r", end="")
    print(f"Brightfield preprocessing: {idx + 1} / {len(stitched_not_stacked)}", end="")